In [26]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [28]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
words = []
classes = []
documents = []
ignore_letters = [",", "!", ".", "?"]

In [31]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)

[(['hello'], 'greetings'), (['hey'], 'greetings'), (['hi'], 'greetings'), (['good', 'day'], 'greetings'), (['what', "'s", 'up'], 'greetings'), (['how', 'is', 'it', 'going', '?'], 'greetings'), (['cya'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'leaving'], 'goodbye'), (['Have', 'a', 'good', 'day'], 'goodbye'), (['bye'], 'goodbye'), (['ciao'], 'goodbye'), (['see', 'yaa'], 'goodbye'), (['how', 'old'], 'age'), (['how', 'old', 'are', 'you'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['age', '?'], 'age'), (['what', 'is', 'your', 'name'], 'name'), (['what', 'should', 'I', 'call', 'you', '?'], 'name'), (['who', 'are', 'you'], 'name'), (['can', 'you', 'tell', 'me', 'your', 'name'], 'name'), (['I', "'d", 'like', 'to', 'buy', 'something'], 'shop'), (['what', 'would', 'you', 'recommend'], 'shop'), (['Best', 'products', 'to', 'buy'], 'shop'), (['what', 'should', 'i', 'buy'], 'shop'), (['buy'], 'shop'), (['What', 'stocks', 'do', 'i', 'own', '?']

In [32]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pk1', 'wb'))
pickle.dump(classes, open('classes.pk1', 'wb'))

print(words)

["'d", "'s", 'Best', 'Goodbye', 'Have', 'I', 'See', 'What', 'a', 'age', 'am', 'are', 'buy', 'bye', 'call', 'can', 'ciao', 'company', 'cya', 'day', 'do', 'going', 'good', 'guy', 'hello', 'hey', 'hi', 'hour', 'how', 'i', 'in', 'invested', 'is', 'it', 'later', 'leaving', 'like', 'me', 'meet', 'my', 'name', 'of', 'old', 'open', 'operation', 'own', 'product', 'recommend', 'see', 'share', 'should', 'something', 'stock', 'tell', 'the', 'timing', 'to', 'up', 'what', 'whats', 'when', 'who', 'would', 'yaa', 'you', 'your']


In [33]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [34]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("Training Completed")

Epoch 1/200


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


8/8 [==============================] - 0s 2ms/step - loss: 1.9615 - accuracy: 0.1653
Epoch 2/200
8/8 [==============================] - 0s 2ms/step - loss: 2.0210 - accuracy: 0.0878
Epoch 3/200
8/8 [==============================] - 0s 2ms/step - loss: 1.8843 - accuracy: 0.1834
Epoch 4/200
8/8 [==============================] - 0s 2ms/step - loss: 1.9247 - accuracy: 0.2063
Epoch 5/200
8/8 [==============================] - 0s 2ms/step - loss: 1.8316 - accuracy: 0.2568
Epoch 6/200
8/8 [==============================] - 0s 2ms/step - loss: 1.6790 - accuracy: 0.4836
Epoch 7/200
8/8 [==============================] - 0s 2ms/step - loss: 1.6757 - accuracy: 0.3750
Epoch 8/200
8/8 [==============================] - 0s 2ms/step - loss: 1.5851 - accuracy: 0.4411
Epoch 9/200
8/8 [==============================] - 0s 2ms/step - loss: 1.5418 - accuracy: 0.5516
Epoch 10/200
8/8 [==============================] - 0s 2ms/step - loss: 1.5495 - accuracy: 0.4382
Epoch 11/200
8/8 [=======================